# Os dados

Lendo e visualizandos os dados do Brasil do COVID até 

In [1]:

import pandas as pd

# Lendo os dados online
data_path = 'https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv'

online_data = pd.read_csv(data_path, delimiter=",") 
online_data.head()


,date,country,state,city,newDeaths,deaths,newCases,totalCases,deathsMS,totalCasesMS,deaths_per_100k_inhabitants,totalCases_per_100k_inhabitants,deaths_by_totalCases,recovered,suspects,tests,tests_per_100k_inhabitants
0,2020-02-25,Brazil,SP,TOTAL,0,0,1,1,0,0,0.0,0.00218,0.0,NaN,NaN,NaN,NaN
1,2020-02-25,Brazil,TOTAL,TOTAL,0,0,1,1,0,0,0.0,0.00048,0.0,NaN,NaN,NaN,NaN
2,2020-02-26,Brazil,SP,TOTAL,0,0,0,1,0,1,0.0,0.00218,0.0,NaN,NaN,NaN,NaN
3,2020-02-26,Brazil,TOTAL,TOTAL,0,0,0,1,0,1,0.0,0.00048,0.0,NaN,NaN,NaN,NaN
4,2020-02-27,Brazil,SP,TOTAL,0,0,0,1,0,1,0.0,0.00218,0.0,NaN,NaN,NaN,NaN


## Filtrando e limpando os dados

In [2]:

at_state = online_data['state']=="TOTAL"
local_data = online_data[at_state]

local_data = local_data[local_data.recovered.notnull()]
local_data.head()


,date,country,state,city,newDeaths,deaths,newCases,totalCases,deathsMS,totalCasesMS,deaths_per_100k_inhabitants,totalCases_per_100k_inhabitants,deaths_by_totalCases,recovered,suspects,tests,tests_per_100k_inhabitants
321,2020-03-23,Brazil,TOTAL,TOTAL,9,34,358,1952,34,1891,0.01618,0.92887,0.01742,8.0,15867.0,NaN,NaN
349,2020-03-24,Brazil,TOTAL,TOTAL,13,47,303,2255,46,2201,0.02237,1.07306,0.02084,20.0,17700.0,NaN,NaN
377,2020-03-25,Brazil,TOTAL,TOTAL,12,59,311,2566,57,2433,0.02808,1.22105,0.02299,27.0,27227.0,NaN,NaN
405,2020-03-26,Brazil,TOTAL,TOTAL,18,77,424,2990,77,2915,0.03664,1.42281,0.02575,42.0,48793.0,NaN,NaN
433,2020-03-27,Brazil,TOTAL,TOTAL,16,93,486,3476,92,3417,0.04425,1.65408,0.02675,42.0,50684.0,NaN,NaN


In [3]:

import numpy as np
from datetime import datetime

only_SP = False

first_date = local_data["date"].iloc[0]
first_date = datetime.fromisoformat(first_date)

if only_SP:
    N = 11869660
else:
    N = 220e6
    
I = list()
R = local_data["recovered"].iloc[1:].to_numpy()  # <- R(t)

M = local_data["newDeaths"].iloc[1:].to_numpy()
nR = np.diff(local_data["recovered"].to_numpy()) #
nC = local_data["newCases"].iloc[1:].to_numpy()

# I(t) = I(t-1) + newCases(t) - newMortes(t) - newRecovered(t)
I = [ local_data["totalCases"].iloc[1] ]

for t in range(len(M)-1):
    I.append(I[-1] + nC[t] - M[t] - nR[t])
I = np.array(I)

    

In [4]:
first_date

datetime.datetime(2020, 3, 23, 0, 0)

## Visualizando a evolução

In [5]:

from bokeh.models   import Legend, ColumnDataSource, RangeTool, LinearAxis, Range1d, HoverTool
from bokeh.palettes import brewer, Inferno256
from bokeh.plotting import figure, show
from bokeh.layouts  import column
from bokeh.io       import output_notebook

output_notebook()

from datetime import timedelta

# Criando o vetor de tempo
date_vec = [ first_date + timedelta(days=k) for k in range(len(M))]

# Criando os valores para legenda no plot
year =  [str(int(d.year)) for d in date_vec ]
month = [("0"+str(int(d.month)))[-2:] for d in date_vec ]
day =   [("0"+str(int(d.day)))[-2:] for d in date_vec ]



# Criando a fonte de dados
source = ColumnDataSource(data={
    'Data'       : date_vec,
    'd': day, 'm': month, 'y': year,
    'Infectados' : I,
    'Removidos'  : R,
    'Mortes'     : M,
})


# Criando a figura
p = figure(plot_height=500,
           plot_width=600,
           x_axis_type="datetime", 
           tools="", 
           toolbar_location=None,
#            y_axis_type="log",
           title="Evolução do COVID - São Paulo")

# Preparando o estilo
p.grid.grid_line_alpha = 0
p.ygrid.band_fill_color = "olive"
p.ygrid.band_fill_alpha = 0.1
p.yaxis.axis_label = "Indivíduos"
p.xaxis.axis_label = "Dias"

# Incluindo as curvas
i_p = p.line(x='Data', y='Infectados', legend_label="Infectados", line_cap="round", line_width=3, color="#ffd885", source=source)
m_p = p.line(x='Data', y='Mortes', legend_label="Mortes", line_cap="round", line_width=3, color="#de425b", source=source)
r_p = p.line(x='Data', y='Removidos', legend_label="Removidos", line_cap="round", line_width=3, color="#99d594", source=source)

# Colocando as legendas
p.legend.click_policy="hide"
p.legend.location = "top_left"

# Incluindo a ferramenta de hover
p.add_tools(HoverTool(
    tooltips=[
        ( 'Indivíduos', '$y{i}'),
        ( 'Data',       '@d/@m/@y' ),
    ],
    renderers=[
        r_p, i_p, m_p
    ]
))

show(p)


Loading BokehJS ...

# O problema

O conjunto de equações diferenciais que caracteriza o modelo é descrito abaixo. No modelo $\beta - \text{representa a taxa de transmissão ou taxa efetiva de contato} $  e $r - \text{a taxa de remoção ou recuperação.}$ 


$$ \begin{split}
   \frac{dS(t)}{dt} & = -\beta S(t) I(t) \\
   \frac{dI(t)}{dt} & = \beta S(t) I(t) - rI(t)  \\
   \frac{dR(t)}{dt} & = r I(t)
   \end{split}$$

   Gostaríamos de identificar quais parâmetros $\beta$ e $r$ resultam num melhor ajuste do modelo para os dados de **S**,**I** e **R**

In [6]:

# Importando o modelo SIR
from models import *

sir_model = ss.SIR(pop=N, focus=["S", "I", "R"])


Loading BokehJS ...

# Estimando os parâmetros

Para estimarmos os parâmetros do modelo $\mathbf{\beta}$ e $\mathbf{r}$, vamos utilizar  inicialmente o método de mínimos quadrados. Podemos então formular o problema a partir da Equação abaixo. Na Equação $y_m(k)$ representa o dado real em cada amostra $k$; $y_s(\theta,k)$ representa o **valor estimado** a partir da simulação do modelo para uma determinada amostra $k$ e $\theta$ representa o vetor ed parâmetros $\theta = [ \beta \; \; r]^T$. 

$$ min_{\theta}= \sum_{k=1}^{K}(y_m(k) - y_s(\theta,k))^2 $$

A equação formula a pergunta: quais os valores de $beta$ e $r$ que minizam o erro quadrático quando comparados com os dados reais.

In [7]:

import numpy as np

S = N - I - R

time = np.linspace(0, len(I), len(I))

# Estimando os parâmetros
sir_model.fit(S, I, R, time, beta_sens=[100,10], r_sens=[1000,10])

r_included = True


	 ├─ S(0) ─ I(0) ─ R(0) ─  [219997725.0, 2255.0, 20.0]
	 ├─ beta ─   1   r ─   0.14285714285714285
	 ├─ beta bound ─   0.01  ─  10
	 ├─ r bound ─   0.00014285714285714284  ─  1.4285714285714284
	 ├─ equation weights ─   [0.0008977501926440376, 1, 1]
	 └─ Defined at:  0.12825878694327464  ─  0.05664658365719824 



In [8]:
sir_model.parameters[0]/sir_model.parameters[1]

2.2641928014484316

In [9]:

if r_included:
    initial = [S[0], I[0], R[0]]
else:
    initial = [S[0], I[0]]
    
results = sir_model.predict(initial, time)


In [10]:

# Incluindo os dados de infectados
im_p = p.line(
    date_vec, results[1],
    legend_label="Infectados - Modelo",
    line_width=4,
    line_dash="dashed",
    line_cap="round",
    color="#f57f17"
)

# Incluindo os dados de recuperados
if r_included:
    rm_p = p.line(
        date_vec, results[2],
        legend_label="Removidos - Modelo",
        line_dash="dashed",
        line_width=4,
        line_cap="round",
        color="#1b5e20"
    )

show(p)


# Predições utilizando o modelo

In [11]:

# Criando os valores de tempo para previsão - 120 dias
t_sim = np.linspace(0, len(I) + 120, len(I) + 120)
date_vec_sim = [first_date + timedelta(days=k) for k in t_sim]

# Prevendo para os valores selecionados
prediction = sir_model.predict(initial, t_sim)



# Criando o gráfico com as predições

# Criando os valores para legenda no plot
year_sim =  [str(int(d.year)) for d in date_vec_sim ]
month_sim = [("0"+str(int(d.month)))[-2:] for d in date_vec_sim ]
day_sim =   [("0"+str(int(d.day)))[-2:] for d in date_vec_sim ]

if r_included:
    accum_Infect = [0]
    for i in N - prediction[1] - prediction[2]:
        accum_Infect.append(accum_Infect[-1]+i)

    accum_Infect = sir_model.parameters[1] * np.array(accum_Infect)
    
# Criando a fonte de dados
if r_included:
    source = ColumnDataSource(data={
        'Data'       : date_vec,
        'd': day, 'm': month, 'y': year,
        'Infectados' : I,
        'Removidos'  : R,
        'Mortes'     : M,
        'InfecModelo' : prediction[1],
        'RemovModelo' : prediction[2],
        'AccumInfect' : accum_Infect,
        'SucetModelo' : N - prediction[1] - prediction[2],
        'DataModelo'  : date_vec_sim,
        'ds': day_sim, 'ms': month_sim, 'ys': year_sim
    })
else:
    source = ColumnDataSource(data={
        'Data'       : date_vec,
        'd': day, 'm': month, 'y': year,
        'Infectados' : I,
        'Removidos'  : R,
        'Mortes'     : M,
        'InfecModelo' : prediction[1],
        'DataModelo'  : date_vec_sim,
        'ds': day_sim, 'ms': month_sim, 'ys': year_sim
    })

    
# Criando a figura
p = figure(plot_height=700,
           plot_width=800,
           x_axis_type="datetime", 
           tools="", 
           toolbar_location=None,
           y_axis_type="log",
           title="Previsão do COVID - Brasil")

# Preparando o estilo
p.grid.grid_line_alpha = 0
p.ygrid.band_fill_color = "olive"
p.ygrid.band_fill_alpha = 0.1
p.yaxis.axis_label = "Indivíduos"
p.xaxis.axis_label = "Dias"

# Incluindo as curvas
i_p = p.line(x='Data', y='Infectados', legend_label="Infectados", line_cap="round", line_width=3, color="#ffd885", source=source)
m_p = p.line(x='Data', y='Mortes', legend_label="Mortes", line_cap="round", line_width=3, color="#de425b", source=source)
r_p = p.line(x='Data', y='Removidos', legend_label="Removidos", line_cap="round", line_width=3, color="#99d594", source=source)

mp_p = p.line(x='DataModelo', y='InfecModelo', legend_label="Infectados - Modelo", line_dash="dashed", line_cap="round", line_width=4, color="#f57f17", source=source)

renders = [i_p, m_p, r_p, mp_p]

if r_included:
    rp_p = p.line(x='DataModelo', y='RemovModelo', legend_label="Removidos - Modelo", line_dash="dashed", line_cap="round", line_width=4, color="#1b5e20", source=source)
    renders.append(rp_p)
    
# Colocando as legendas
p.legend.click_policy="hide"
p.legend.location = "top_left"

# Incluindo a ferramenta de hover
p.add_tools(HoverTool(
    tooltips=[
        ( 'Indivíduos', '$y{0.00 a}' ),
        ( 'Data',       '@ds/@ms/@ys'),
    ],
    renderers=renders
))

show(p)


# Referências


- [Predictive Monitoring of COVID-19](https://ddi.sutd.edu.sg)

- [Apple mobility data](https://www.apple.com/covid19/mobility)

- [Corona Virus - Brazil Data](https://painel.covid19br.org)

- [Fitting model to Corona Virus](https://towardsdatascience.com/infectious-disease-modelling-fit-your-model-to-coronavirus-data-2568e672dbc7)